In [5]:
using Revise

using Kate

### Check Environment

In [6]:
Kate.test()

Testing environment...
`which skewer`
/opt/conda/bin/skewer
`which fastqc`
/opt/conda/bin/fastqc
`which bgzip`
/opt/conda/bin/bgzip
`which tabix`
/opt/conda/bin/tabix
`which minimap2`
/opt/conda/bin/minimap2
`which samtools`
/opt/conda/bin/samtools
`which bcftools`
/opt/conda/bin/bcftools
`which kallisto`
/opt/conda/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/opt/conda/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py


### Set Parameters

In [7]:
# Sample name
sa = "2012_cat"

# Molecule: cdna or dna
mo = "dna"

# Adapter sequence to trim
ad = "AGATCGGAAGAGC"

# Forward read file
fq1 = "../input/2012_cat/2012_R1.fastq.gz"

# Reverse read file
fq2 = "../input/2012_cat/2012_R2.fastq.gz"

# is dna targeted
ta = false

# Number of jobs
n_jo = 10

# G of memory
met = 40

# G of memory per job
mej = 2;

In [8]:
pap = dirname(@__DIR__)

pao = joinpath(pap, "output")

pai = joinpath(pap, "input")

par = joinpath(pai, "human_reference_genome")

"/home/jovyan/Kate.jl/input/human_reference_genome"

### Process DNA

In [ ]:
Kate.process_dna(
    mo,
    fq1,
    fq2,
    ad,
    ta,
    joinpath(pao, string(sa)),
    joinpath(par, "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz"),
    joinpath(par, "chromosome.bed.gz"),
    joinpath(par, "chrn_n.tsv"),
    "/opt/snpeff/snpEff/snpEff.jar",
    n_jo,
    met,
    mej,
)

(2021-08-21T05:31:19.982) Trimming sequence ...
Made path for trimmed files: /home/jovyan/Kate.jl/output/2012_cat/trim/
`skewer --threads 10 -x AGATCGGAAGAGC --mode pe --mean-quality 10 --end-quality 15 --compress --output /home/jovyan/Kate.jl/output/2012_cat/trim/ --quiet ../input/2012_cat/2012_R1.fastq.gz ../input/2012_cat/2012_R2.fastq.gz`
.--. .-.
: .--': :.-.
`. `. : `'.' .--. .-..-..-. .--. .--.
_`, :: . `.' '_.': `; `; :' '_.': ..'
`.__.':_;:_;`.__.'`.__.__.'`.__.':_;
skewer v0.2.2 [April 4, 2016]
Parameters used:
-- 3' end adapter sequence (-x):	AGATCGGAAGAGC
-- maximum error ratio allowed (-r):	0.100
-- maximum indel error ratio allowed (-d):	0.030
-- mean quality threshold (-Q):		10
-- end quality threshold (-q):		15
-- minimum read length allowed after trimming (-l):	18
-- file format (-f):		Sanger/Illumina 1.8+ FASTQ (auto detected)
-- number of concurrent threads (-t):	10
Sat Aug 21 05:31:20 2021 >> started

Sat Aug 21 23:01:53 2021 >> done (63104.766s)
1987819836 read pai

Started analysis of trimmed-pair1.fastq.gz
Started analysis of trimmed-pair2.fastq.gz
Approx 5% complete for trimmed-pair1.fastq.gz
Approx 5% complete for trimmed-pair2.fastq.gz
Approx 10% complete for trimmed-pair1.fastq.gz
Approx 10% complete for trimmed-pair2.fastq.gz
Approx 15% complete for trimmed-pair1.fastq.gz
Approx 15% complete for trimmed-pair2.fastq.gz
Approx 20% complete for trimmed-pair1.fastq.gz
Approx 20% complete for trimmed-pair2.fastq.gz
Approx 25% complete for trimmed-pair1.fastq.gz
Approx 25% complete for trimmed-pair2.fastq.gz
Approx 30% complete for trimmed-pair1.fastq.gz
Approx 30% complete for trimmed-pair2.fastq.gz
Approx 35% complete for trimmed-pair1.fastq.gz
Approx 35% complete for trimmed-pair2.fastq.gz
Approx 40% complete for trimmed-pair1.fastq.gz
Approx 40% complete for trimmed-pair2.fastq.gz
Approx 45% complete for trimmed-pair1.fastq.gz
Approx 45% complete for trimmed-pair2.fastq.gz
Approx 50% complete for trimmed-pair1.fastq.gz
Approx 50% complete for

Analysis complete for trimmed-pair1.fastq.gz
Analysis complete for trimmed-pair2.fastq.gz
Checking sequence bias ...
`multiqc --outdir /home/jovyan/Kate.jl/output/2012_cat/check_trim /home/jovyan/Kate.jl/output/2012_cat/check_trim`



  /// MultiQC 🔍 | v1.11

|           multiqc | Search path : /home/jovyan/Kate.jl/output/2012_cat/check_trim


|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 4/4  

|            fastqc | Found 2 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : ../output/2012_cat/check_trim/multiqc_report.html
|           multiqc | Data        : ../output/2012_cat/check_trim/multiqc_data
|           multiqc | MultiQC complete


Done at 2021-08-22T04:13:25.266 in 5 hours, 11 minutes, 31 seconds, 580 milliseconds.

(2021-08-22T04:13:25.296) Aligning sequence ...
pipeline(pipeline(pipeline(pipeline(`minimap2 -x sr -t 10 -K 2G -R '@RG\tID:Germ\tSM:Germ' -a /home/jovyan/Kate.jl/input/human_reference_genome/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz.mmi /home/jovyan/Kate.jl/output/2012_cat/trim//trimmed-pair1.fastq.gz /home/jovyan/Kate.jl/output/2012_cat/trim//trimmed-pair2.fastq.gz`, stdout=`samtools sort --threads 10 -n`), stdout=`samtools fixmate --threads 10 -m - -`), stdout=`samtools sort --threads 10`), stdout>Base.FileRedirect("/home/jovyan/Kate.jl/output/2012_cat/align/germ.bam.tmp", false))


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::40.460*0.36] loaded/built the index for 195 target sequence(s)
[M::mm_mapopt_update::40.461*0.36] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 195
[M::mm_idx_stat::41.611*0.38] distinct minimizers: 100167746 (38.80% are singletons); average occurrences: 5.519; average spacing: 5.607; total length: 3099922541
[M::worker_pipeline::691.829*7.91] mapped 13342492 sequences
[M::worker_pipeline::1255.790*8.80] mapped 13343072 sequences
[M::worker_pipeline::1618.730*9.14] mapped 13343198 sequences
[M::worker_pipeline::2190.917*9.31] mapped 13343646 sequences
[M::worker_pipeline::2704.420*9.41] mapped 13343584 sequences
[M::worker_pipeline::3136.025*9.54] mapped 13343816 sequences
[M::worker_pipeline::3907.628*9.58] mapped 13343758 sequences
[M::worker_pipeline::4491.535*9.57] mapped 13343620 sequences
[M::worker_pipeline::4896.307*9.64] mapped 13343254 sequen

## Benchmark

TODO: Refactor

In [ ]:
# baseline vcf
vb = joinpath(pai, "high_confidence_HG002_4.1_GRCh38/HG002_GRCh38_1_22_v4.1_draft_benchmark.vcf.gz")

# calls vcf
vc = joinpath(pao, "HG002_cat/find_variant/strelka/results/variants/variants.vcf.gz")

sdf = joinpath(pai, "GRCh38.sdf")

pab = joinpath(pao, "HG002_cat/benchmark")

rtg = "/opt/rtg/rtg-tools-3.11/rtg"

### Configure vcfeval

In [ ]:
run(`$rtg`)

### Download Real Time Genomics prepard GRCH38 files
vcfeval requires reference files in RTG formats. RTG makes versions of these files for the popular reference genomes such as GRCh38, which will be downloaded below.

In [ ]:
if ispath(sdf)
    
    println("Skipping download because RTG GRCh38 data directory already exists:\n $sdf")

else

    run(pipeline(
            `wget -q -P ../input/ https://s3.amazonaws.com/rtg-datasets/references/GRCh38.sdf.zip`,
            `unzip -d ../input/ ../input/GRCh38.sdf.zip`,
            `rm ../input/GRCh38.sdf.zip`,
            ))
end

### Run vcfeval
This command compares the VCF you generated (calls) to the highly confident VCF generated by Genome in a Bottle (baseline).

In [ ]:
run(`$rtg vcfeval
    --baseline=$vb
    --calls=$vc
    --template=$sdf
    --output=$pab
    --threads=1
    `)

### See results
Read more about how to interpret vcfeval results [here](https://cdn.rawgit.com/RealTimeGenomics/rtg-tools/master/installer/resources/tools/RTGOperationsManual/rtg_command_reference.html#vcfeval).

In [ ]:
# vcfeval summary file
pasu = open(joinpath(pab, "summary.txt"))

su = read(pasu)

su